In [12]:
%matplotlib inline
import numpy as np
import os
import os.path as osp
import gym
import imageio
import matplotlib.pyplot as plt

from torch import nn
from IPython.display import Image, Video
from gym import wrappers
from rrc_iprl_package.envs import cube_env, custom_env, rrc_utils
from spinup.utils import test_policy

from spinup import ppo_pytorch as ppo

In [2]:
custom_env.PushCubeEnv.observation_names = ['robot_position',
 'robot_velocity',
 'robot_tip_positions',
 'object_position',
 'object_orientation',
 'goal_object_position',
 'goal_object_orientation',           
 'action']

env = rrc_utils.p2_reorient_env_fn()
env.observation_space.shape

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(50,)

In [5]:
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
env = VecNormalize(DummyVecEnv([rrc_utils.p2_reorient_env_fn]), norm_obs=True, norm_reward=False,
                         clip_obs=10., gamma=0.95)

In [8]:
env.step([env.action_space.sample()])

(array([[-8.83503890e-04,  7.04794423e-03, -8.79800358e-03,
          3.36587927e-05,  7.02921541e-03, -8.51769260e-03,
         -4.09008380e-04,  7.00508193e-03, -8.49040298e-03,
         -7.79421815e-06, -4.51033982e-06,  6.69425416e-03,
         -3.97606771e-03,  4.26194565e-04,  9.37645411e-04,
          1.07931447e-04,  8.95752098e-06,  9.25094314e-07,
         -5.41926861e-03, -6.07196186e-03, -4.26194565e-04,
         -9.37645411e-04, -1.07931447e-04, -2.53002591e-05,
          6.82208136e-03, -8.82396122e-03,  2.69288995e-04,
          6.71694084e-03, -8.50742269e-03, -1.17987879e-04,
          6.67224004e-03, -8.47777309e-03,  8.47381853e-04,
          4.74569092e-04,  9.88913157e-04,  2.15573724e-04,
         -1.02616943e-03,  6.59534339e-04, -1.03927186e-03,
          4.03270774e-04,  6.68139216e-04, -9.05647752e-03,
          3.26497222e-03, -7.26529648e-03,  4.65314076e-03,
          8.71850855e-03, -8.48335027e-03, -2.06070670e-03,
          8.86658933e-03, -9.20944106e-0

In [9]:
env_fn = rrc_utils.p2_reorient_env_fn
ac_kwargs = dict(hidden_sizes=[64,64], activation=nn.Tanh)
logger_kwargs = dict(output_dir='./data/ppo_rrc_cube', exp_name='cube_rrc')

In [10]:
# load_path = '/scr1/Developer/Projects/spinningup/data/td3_rrc_t1/td3_rrc_t1_s0'
# exp_name = osp.split(load_path.rstrip('/'))[1]
exp_name = 'ppo-reorient'

vid_dir = osp.join('./videos', exp_name)
if not osp.exists(vid_dir):
    os.makedirs(vid_dir)

i = 0
vid_save_path = lambda: osp.join(vid_dir, 'ep_{}.mp4'.format(i))

while osp.exists(vid_save_path()): i +=1 
vid_save_path = vid_save_path()

In [51]:
env, policy = test_policy.load_policy_and_env(load_path, 'last', True)

if env is None:
    env = rrc_utils.rrc_ppo_env_fn()



Loading from /scr1/Developer/Projects/spinningup/data/td3_rrc_t1/td3_rrc_t1_s0/pyt_save/model.pt.




In [53]:
%%time
obs = env.reset()
env.unwrapped.platform.enable_cameras = True
env.render = lambda: env.unwrapped.platform.get_camera_observation(env.step_count).cameras[1].image
d = False
# frames = [env.render()]

while not d:
    obs, r, d, i = env.step(policy(obs))
#     frames.append(env.render())

In [47]:
with imageio.get_writer(vid_save_path) as writer:
    for frame in frames:
        writer.append_data(frame)

12

In [22]:
obs

{'robot_position': array([ 0.        ,  1.22173047, -2.26892805,  0.        ,  1.22173047,
        -2.26892805,  0.        ,  1.22173047, -2.26892805]),
 'robot_velocity': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'robot_tip_positions': array([[ 0.08457   ,  0.06003509,  0.15397678],
        [ 0.00970691, -0.10325731,  0.15397678],
        [-0.09427691,  0.04322223,  0.15397678]]),
 'object_position': array([ 0.01153694, -0.06141343,  0.0325    ]),
 'object_orientation': array([ 0.        ,  0.        ,  0.93348761, -0.35860964]),
 'goal_object_position': array([-0.00781044,  0.1087707 ,  0.0325    ])}

In [13]:
ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=250, logger_kwargs=logger_kwargs,
    info_kwargs=rrc_utils.p2_info_kwargs)

Logging data to ./data/ppo_rrc_cube/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"Tanh",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.2,
    "early_stopping_fn":	null,
    "env_fn":	"env_fn",
    "epochs":	250,
    "exp_name":	"cube_rrc",
    "gamma":	0.99,
    "info_kwargs":	{},
    "lam":	0.97,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7f1c7cf7f588>":	{
            "epoch_dict":	{},
            "exp_name":	"cube_rrc",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "max_saves":	10,
            "output_dir":	"./data/ppo_rrc_cube",
            "output_file":	{
                "<_io.TextIOWrapper name='./data/ppo_rrc_cube/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/scr-ssd/ksrini/trifinger_simulation/python/trifinger_simulation/trifinger_platform.py:291: UserWarning: Cameras are not enabled, so images in the camera observation are not initialized.  Create `TriFingerPlatform` with `enable_cameras=True` to get rendered camera images.
  "Cameras are not enabled, so images in the camera observation"


Early stopping at step 29 due to reaching max kl.
---------------------------------------
|             Epoch |               0 |
|      AverageEpRet |            -185 |
|          StdEpRet |             296 |
|          MaxEpRet |             292 |
|          MinEpRet |            -635 |
|             EpLen |             500 |
|      AverageVVals |           0.255 |
|          StdVVals |          0.0932 |
|          MaxVVals |           0.588 |
|          MinVVals |          -0.137 |
| TotalEnvInteracts |           5e+03 |
|            LossPi |        1.68e-08 |
|             LossV |        9.05e+03 |
|       DeltaLossPi |         -0.0225 |
|        DeltaLossV |            -469 |
|           Entropy |           0.919 |
|                KL |          0.0151 |
|          ClipFrac |           0.203 |
|          StopIter |              29 |
|              Time |            67.1 |
---------------------------------------


/scr-ssd/ksrini/trifinger_simulation/python/trifinger_simulation/trifinger_platform.py:291: UserWarning: Cameras are not enabled, so images in the camera observation are not initialized.  Create `TriFingerPlatform` with `enable_cameras=True` to get rendered camera images.
  "Cameras are not enabled, so images in the camera observation"


Early stopping at step 56 due to reaching max kl.
---------------------------------------
|             Epoch |               1 |
|      AverageEpRet |            -340 |
|          StdEpRet |             269 |
|          MaxEpRet |           -33.3 |
|          MinEpRet |            -865 |
|             EpLen |             500 |
|      AverageVVals |           -8.52 |
|          StdVVals |          0.0381 |
|          MaxVVals |           -8.18 |
|          MinVVals |           -8.59 |
| TotalEnvInteracts |           1e+04 |
|            LossPi |        -9.3e-10 |
|             LossV |        7.74e+03 |
|       DeltaLossPi |         -0.0249 |
|        DeltaLossV |            -305 |
|           Entropy |           0.919 |
|                KL |           0.015 |
|          ClipFrac |            0.21 |
|          StopIter |              56 |
|              Time |             134 |
---------------------------------------
Early stopping at step 6 due to reaching max kl.
---------------------

...

Early stopping at step 6 due to reaching max kl.
---------------------------------------
|             Epoch |             221 |
|      AverageEpRet |           -17.7 |
|          StdEpRet |             326 |
|          MaxEpRet |             498 |
|          MinEpRet |            -706 |
|             EpLen |             500 |
|      AverageVVals |            30.2 |
|          StdVVals |            34.7 |
|          MaxVVals |             160 |
|          MinVVals |            -121 |
| TotalEnvInteracts |        1.11e+06 |
|            LossPi |       -1.32e-08 |
|             LossV |        1.38e+04 |
|       DeltaLossPi |         -0.0122 |
|        DeltaLossV |       -6.61e+03 |
|           Entropy |           0.791 |
|                KL |          0.0155 |
|          ClipFrac |           0.201 |
|          StopIter |               6 |
|              Time |        1.35e+04 |
---------------------------------------
Early stopping at step 6 due to reaching max kl.
----------------------

/scr-ssd/ksrini/trifinger_simulation/python/trifinger_simulation/trifinger_platform.py:291: UserWarning: Cameras are not enabled, so images in the camera observation are not initialized.  Create `TriFingerPlatform` with `enable_cameras=True` to get rendered camera images.
  "Cameras are not enabled, so images in the camera observation"


Early stopping at step 11 due to reaching max kl.
---------------------------------------
|             Epoch |             231 |
|      AverageEpRet |            -147 |
|          StdEpRet |             288 |
|          MaxEpRet |             273 |
|          MinEpRet |            -747 |
|             EpLen |             500 |
|      AverageVVals |           -1.04 |
|          StdVVals |            25.6 |
|          MaxVVals |             177 |
|          MinVVals |           -55.7 |
| TotalEnvInteracts |        1.16e+06 |
|            LossPi |        2.98e-09 |
|             LossV |        1.16e+04 |
|       DeltaLossPi |         -0.0177 |
|        DeltaLossV |       -4.33e+03 |
|           Entropy |           0.787 |
|                KL |          0.0153 |
|          ClipFrac |           0.232 |
|          StopIter |              11 |
|              Time |        1.42e+04 |
---------------------------------------
Early stopping at step 7 due to reaching max kl.
---------------------

/scr-ssd/ksrini/trifinger_simulation/python/trifinger_simulation/trifinger_platform.py:291: UserWarning: Cameras are not enabled, so images in the camera observation are not initialized.  Create `TriFingerPlatform` with `enable_cameras=True` to get rendered camera images.
  "Cameras are not enabled, so images in the camera observation"


Early stopping at step 6 due to reaching max kl.
---------------------------------------
|             Epoch |             241 |
|      AverageEpRet |            -149 |
|          StdEpRet |             619 |
|          MaxEpRet |             592 |
|          MinEpRet |        -1.7e+03 |
|             EpLen |             500 |
|      AverageVVals |           -22.6 |
|          StdVVals |            30.8 |
|          MaxVVals |            71.6 |
|          MinVVals |            -102 |
| TotalEnvInteracts |        1.21e+06 |
|            LossPi |        1.12e-08 |
|             LossV |        1.82e+04 |
|       DeltaLossPi |         -0.0121 |
|        DeltaLossV |       -6.97e+03 |
|           Entropy |           0.786 |
|                KL |          0.0157 |
|          ClipFrac |           0.216 |
|          StopIter |               6 |
|              Time |        1.48e+04 |
---------------------------------------
Early stopping at step 5 due to reaching max kl.
----------------------